In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
findspark.find()

'/Users/avinashs/Downloads/softwares/spark-3.0.0-bin-hadoop2.7'

In [4]:
from pyspark.sql import SparkSession

### Spark Session object

In [6]:
spark = SparkSession.builder.appName("Anaconda python").master("local[5]").getOrCreate()

22/08/13 07:26:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark

### Create Spark Context from Spark session object

In [8]:
sc = spark.sparkContext

### Create rdd from memory

In [ ]:
numbers_rdd = sc.parallelize(range(1,101), 4)

In [ ]:
numbers_rdd

## Map partitions with Index

In [ ]:
def print_elements_in_partition(index, elements):
    for element in elements:
        yield f"partition: {index} has element: {element}"

In [ ]:
numbers_rdd.mapPartitionsWithIndex(print_elements_in_partition).collect()

### Map transfromation

In [ ]:
numbers_rdd.map(lambda x: x*x).collect()

### Filter transformation

In [ ]:
numbers_rdd.filter(lambda x : x%2 ==0).collect()

### Get Number of partitions of a rdd

In [ ]:
numbers_rdd.getNumPartitions()

### Count Action

In [ ]:
numbers_rdd.count()

### flatMap transformation

In [ ]:
sentences = ["This is a sentence1.", "This is sentence2.", "This is sentence3"]

In [ ]:
sentence_rdd = sc.parallelize(sentences)

In [ ]:
sentence_rdd.collect()

In [ ]:
sentence_rdd.flatMap(lambda x:x.split()).collect()

In [ ]:
rdd = sc.textFile("/Users/avinashs/Downloads/pyspark/datasets/Baby_Names__Beginning_2007.csv")

In [ ]:
rdd

In [ ]:
rdd.getNumPartitions()

In [ ]:
rdd.collect()

In [ ]:
rdd.count()

In [ ]:
def count_baby_names_partitions_wise(index, iterator):
    count_records = 0
    for row in iterator:
        count_records += 1
    yield f"Parition {index} has {count_records} records"

In [ ]:
rdd.mapPartitionsWithIndex(count_baby_names_partitions_wise).collect()

In [ ]:
def skip_header(index, iterator):
    number = 0
    for row in iterator:
        if index == 0 and number == 0:
            number += 1
            continue
        yield row
            
    
            

In [ ]:
data_rdd = rdd.mapPartitionsWithIndex(skip_header)

In [ ]:
year_rdd = data_rdd.map(lambda x: (x.split(",")[0], int(x.split(",")[-1])))

In [ ]:
year_rdd.reduceByKey(lambda x, y : x+y).collect()

In [ ]:
year_rdd.groupByKey().mapValues(sum).collect()

In [ ]:
year_rdd.getNumPartitions()

### Increase the nuber of partition

In [ ]:
year_rdd = year_rdd.repartition(4)

In [ ]:
year_rdd.getNumPartitions()

### Decrease the number of partition

In [ ]:
year_rdd.coalesce(3).getNumPartitions()

### Write rdd to a text file

In [ ]:
year_rdd.saveAsTextFile("years.txt")

In [ ]:
rdd_repartition = rdd.repartition(4)

In [ ]:
rdd_repartition.collect()

In [9]:
file_rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/ebook.txt", 4)

In [ ]:
file_rdd.collect()

In [ ]:
file_rdd.flatMap(lambda x: x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda x,y: x+y).collect()

In [10]:
filename = "/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/weather.csv"
file1_rdd = sc.textFile(filename)

### take action

In [11]:
file1_rdd.take(5)

['2016-05-09,234893,34',
 '2019-09-08,234896,3',
 '2019-11-19,234895,24',
 '2017-04-04,234900,43',
 '2013-12-04,234900,47']

In [12]:
file1_rdd.first()

'2016-05-09,234893,34'

In [15]:
file1_rdd.map(lambda x: (x.split(",")[0], int(x.split(",")[-1]))).map(lambda x: (int(x[0].split("-")[0]), x[1])).reduceByKey(lambda x, y: max(x,y)).sortByKey().collect()

[(2010, 39),
 (2011, 38),
 (2012, 40),
 (2013, 47),
 (2014, 35),
 (2015, 41),
 (2016, 36),
 (2017, 47),
 (2018, 45),
 (2019, 47)]

### Accumulators

In [16]:
accum = sc.accumulator(0)

In [17]:
file_rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/ebook.txt")

In [19]:
file_rdd.getNumPartitions()

2

In [35]:
def count_blank_lines_in_partition(iterator):
    count_blank_lines = 0
    for row in iterator:
        if len(row.strip()) == 0:
            count_blank_lines += 1

    accum.add(count_blank_lines)

In [37]:
file_rdd.foreachPartition(count_blank_lines_in_partition)

In [38]:
accum.value

539